## Project-Computer Vision with CNN [Plant_Seed_CNN Project1.ipynb]

1. Read the images and generate the train and test dataset (10 points) 

In [0]:
%tensorflow_version 2.x

In [0]:
from google.colab import drive
import os
from zipfile import ZipFile
import cv2
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras import applications
from tensorflow.keras.models import Sequential, Model 
from tensorflow.keras.layers import Dropout, Flatten, Dense
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import SGD , Adam
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.layers import BatchNormalization
import tensorflow
tensorflow.random.set_seed(42)

In [0]:
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
os.chdir('/content/drive/My Drive/')

In [0]:
with ZipFile('/content/drive/My Drive/plant-seedlings-classification.zip', 'r') as z:
  z.extractall()

In [0]:
x_test=[]
os.chdir('/content/drive/My Drive/test')
for i in os.listdir():
    dummy = cv2.imread(i)
    dummy = cv2.resize(dummy,(128,128))
    x_test.append(dummy)

In [0]:
X_test = np.array(x_test)
X_test.shape

(794, 128, 128, 3)

In [0]:
train_dir = '/content/drive/My Drive/train'
os.chdir(train_dir)
os.listdir()

['Black-grass',
 'Charlock',
 'Cleavers',
 'Common Chickweed',
 'Common wheat',
 'Fat Hen',
 'Loose Silky-bent',
 'Maize',
 'Scentless Mayweed',
 'Shepherds Purse',
 'Small-flowered Cranesbill',
 'Sugar beet',
 'Plant_seed_CNN_checkpoint.h5']

In [0]:
x_train = []
Y_train = []
import cv2
for i in os.listdir():
    print(i)
    if (os.path.isdir(i)):
            for j in os.listdir(i):
                try:
                    dummy = cv2.imread(train_dir+"/" + i + "/" + j)
                    dummy = cv2.resize(dummy,(128,128))
                    x_train.append(dummy)
                    Y_train.append(i)
                except Exception as e:
                    print(e)

Black-grass
Charlock
Cleavers
Common Chickweed
Common wheat
Fat Hen
Loose Silky-bent
Maize
Scentless Mayweed
Shepherds Purse
Small-flowered Cranesbill
Sugar beet


In [0]:
y_dummies = pd.get_dummies(Y_train)

In [0]:
y_dummies.head(1)

,Black-grass,Charlock,Cleavers,Common Chickweed,Common wheat,Fat Hen,Loose Silky-bent,Maize,Scentless Mayweed,Shepherds Purse,Small-flowered Cranesbill,Sugar beet
0,1,0,0,0,0,0,0,0,0,0,0,0


In [0]:
X_train = np.array(x_train)
y_train = np.array(y_dummies)
print("X_train.shape:", X_train.shape)
print("y_train.shape:",y_train.shape)

X_train.shape: (4750, 128, 128, 3)
y_train.shape: (4750, 12)


2.Divide the data set into Train and validation data sets

In [0]:

trainX, valX, trainY, valY = train_test_split(X_train, y_train, test_size = 0.3, random_state=2)
print (len(trainX))
print (len(valX))

3325
1425


In [0]:
trainX = trainX/255.
valX = valX /255.

In [0]:
print ('trainX.shape:',trainX.shape)
print ('valX.shape:',valX.shape)
print ('trainY.shape:',trainY.shape)
print ('valY.shape:',valY.shape)

trainX.shape: (3325, 128, 128, 3)
valX.shape: (1425, 128, 128, 3)
trainY.shape: (3325, 12)
valY.shape: (1425, 12)


3.Initialize & build the model (20 points)

In [0]:
model = Sequential()
model.add(Conv2D(filters=64, kernel_size=(3, 3), input_shape=(128, 128, 3), activation='relu'))
model.add(BatchNormalization(axis=3))
model.add(Conv2D(filters=64, kernel_size=(4, 4), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(BatchNormalization(axis=3))
model.add(Dropout(0.1))
model.add(Conv2D(filters=128, kernel_size=(5, 5), activation='relu'))
model.add(BatchNormalization(axis=3))
model.add(Conv2D(filters=256, kernel_size=(5, 5), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(BatchNormalization(axis=3))
model.add(Dropout(0.1))
model.add(Conv2D(filters=256, kernel_size=(4, 4), activation='relu'))
model.add(BatchNormalization(axis=3))
model.add(Conv2D(filters=512, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(BatchNormalization(axis=3))
model.add(Dropout(0.1))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(12, activation='softmax'))

4.Optimize the model (16 points)

In [0]:
optimizer = Adam(lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer= optimizer, metrics=['accuracy'])
model.summary()

Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_115 (Conv2D)          (None, 126, 126, 64)      1792      
_________________________________________________________________
batch_normalization_133 (Bat (None, 126, 126, 64)      256       
_________________________________________________________________
conv2d_116 (Conv2D)          (None, 123, 123, 64)      65600     
_________________________________________________________________
max_pooling2d_58 (MaxPooling (None, 61, 61, 64)        0         
_________________________________________________________________
batch_normalization_134 (Bat (None, 61, 61, 64)        256       
_________________________________________________________________
dropout_82 (Dropout)         (None, 61, 61, 64)        0         
_________________________________________________________________
conv2d_117 (Conv2D)          (None, 57, 57, 128)     

5.Predict the accuracy for both train and validation data (14 points)

In [0]:
checkpoint = ModelCheckpoint("Plant_seed_CNN_checkpoint_v5.h5", monitor='val_accuracy', verbose=1, save_best_only=True, mode='auto')
early = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=10, verbose=1, mode='auto')
epochs=100
model.fit(trainX, trainY, epochs = epochs, validation_data=(valX, valY), callbacks = [checkpoint, early])

Train on 3325 samples, validate on 1425 samples
Epoch 1/100
3296/3325 [============================>.] - ETA: 0s - loss: 1.7302 - accuracy: 0.4666
Epoch 00001: val_accuracy improved from -inf to 0.13333, saving model to Plant_seed_CNN_checkpoint_v5.h5
3325/3325 [==============================] - 13s 4ms/sample - loss: 1.7223 - accuracy: 0.4695 - val_loss: 18.6109 - val_accuracy: 0.1333
Epoch 2/100
3296/3325 [============================>.] - ETA: 0s - loss: 0.9639 - accuracy: 0.6796
Epoch 00002: val_accuracy improved from 0.13333 to 0.13614, saving model to Plant_seed_CNN_checkpoint_v5.h5
3325/3325 [==============================] - 12s 4ms/sample - loss: 0.9646 - accuracy: 0.6797 - val_loss: 9.3933 - val_accuracy: 0.1361
Epoch 3/100
3296/3325 [============================>.] - ETA: 0s - loss: 0.7047 - accuracy: 0.7646
Epoch 00003: val_accuracy improved from 0.13614 to 0.14316, saving model to Plant_seed_CNN_checkpoint_v5.h5
3325/3325 [==============================] - 10s 3ms/sample -

In [0]:
wights_file = 'Plant_seed_CNN_checkpoint_v5.h5' # choose the best checkpoint 
model.load_weights(wights_file) # load it
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [0]:
score_train = model.evaluate(trainX, trainY)
score_val = model.evaluate(valX, valY)
print('Train accuracy:', round(score_train[1]*100,2), "%")
print('Validation accuracy:', round(score_val[1]*100,2),"%")

1425/1425 [==============================] - 1s 866us/sample - loss: 0.4503 - accuracy: 0.8744
Train accuracy: 99.97 %
Validation accuracy: 87.44 %


Model prediction and Submisssion file creation

In [0]:
predictions = model.predict(X_test)
predictions[0]

array([0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.], dtype=float32)

In [0]:
lable_df = pd.DataFrame({"lable": np.argmax(predictions, axis=1, out=None)})

In [0]:
test = []
for file in os.listdir('/content/drive/My Drive/test'):
    test.append(file)
test = pd.DataFrame(test, columns=['file'])
test.shape

(794, 1)

In [0]:
CATEGORIES = y_dummies.columns

In [0]:
test['species'] = [CATEGORIES[c] for c in lable_df['lable'] ]


In [0]:
test.to_csv('submission.csv', index=False)